# Solving Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

# Prerequisites

In [ ]:
# # Install the OpenAI Python package.
%pip install -U openai "pandas<2.2.3" openpyxl


In [ ]:
# IMPORTS
from IPython.display import display
from openpyxl import load_workbook, Workbook
from typing import Union
from enum import Enum
from openai import OpenAI

import os
import pandas as pd
import time as clock
import re
import random
import requests

### Configure OpenAI API Key.

**IMPORTANT: If you are going to use our custom API Key then make sure that you also use custom base URL as in example below. Otherwise it will not work.**

In [ ]:
# Code example of OpenAI communication

client = OpenAI(
    # In order to use provided API key, make sure that models you create point to this custom base URL.
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    # The temporary API key giving access to ChatGPT 4o model. Quotas apply: you have 500'000 input and 500'000 output tokens, use them wisely ;)
    api_key='a0BIj000001iX7PMAU'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

In [ ]:
class Spreadsheet(object):
    def __init__(self, filename: str = None):
        if filename is None:
            filename = "source.xlsx"
        self.filename: str = f"{os.getcwd()}\\{filename}"
        # Load the workbook if it exists, otherwise create a new one
        try:
            self.workbook = load_workbook(self.filename)
        except FileNotFoundError:
            self.workbook = Workbook()
            self.workbook.save(self.filename)
        self.worksheets = self.workbook.sheetnames  # List of worksheet names

    # List all available worksheets
    def list_worksheets(self):
        return self.workbook.sheetnames

    # Adds a new worksheet with given columns
    def add_worksheet(self, sheet_name: str, columns: list[str]):
        if sheet_name in self.workbook.sheetnames:
            raise ValueError(f"Worksheet '{sheet_name}' already exists")
        ws = self.workbook.create_sheet(title=sheet_name)
        ws.append(columns)  # Add columns as the first row
        self.workbook.save(self.filename)

    # Deletes a worksheet
    def del_worksheet(self, sheet_name: str):
        if sheet_name not in self.workbook.sheetnames:
            print(f"Worksheet '{sheet_name}' does not exist")
        else:
            del self.workbook[sheet_name]
        self.workbook.save(self.filename)

    # Adds a record to the given worksheet
    def add_record(self, sheet_name: str, record: tuple):
        if sheet_name not in self.workbook.sheetnames:
            raise ValueError(f"Worksheet '{sheet_name}' does not exist")

        ws = self.workbook[sheet_name]

        # Check if the record exists (assuming the first element of the record is the unique key)
        record_key = record[0]  # Assuming the first element is the unique identifier
        found = False

        for row in ws.iter_rows(min_row=2, values_only=False):  # Assuming first row contains headers
            if row[0].value == record_key:
                found = True
                for i, cell in enumerate(row):
                    cell.value = record[i]  # Overwrite with the new record values
                break

        if not found:
            ws.append(record)  # Append as a new record if no matching record is found

        self.workbook.save(self.filename)

    # Deletes a record from the given worksheet based on the row index (1-based index)
    def del_record(self, sheet_name: str, record_index: int):
        if sheet_name not in self.workbook.sheetnames:
            raise ValueError(f"Worksheet '{sheet_name}' does not exist")
        ws = self.workbook[sheet_name]
        ws.delete_rows(record_index)  # Delete the row at the specified index
        self.workbook.save(self.filename)

    # Returns all worksheets (or selected ones) as a dictionary of pandas DataFrames
    def as_dataframe(self, worksheets=None):
        if worksheets is None:
            worksheets = self.workbook.sheetnames
        dfs = {}
        for sheet in worksheets:
            if sheet not in self.workbook.sheetnames:
                raise ValueError(f"Worksheet '{sheet}' does not exist")
            ws = self.workbook[sheet]
            data = ws.values
            columns = next(data)  # First row is the columns
            df = pd.DataFrame(data, columns=columns)
            dfs[sheet] = df
        return dfs

In [ ]:
DOC_ID = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
LINK = f"https://docs.google.com/spreadsheets/d/{DOC_ID}/export?format=xlsx"
resp = requests.get(LINK)
FILEPATH = os.getcwd() + f"\\spreadsheet.xlsx"
display(FILEPATH)
while not os.path.exists(FILEPATH):
    with open(FILEPATH, 'wb') as file:
        file.write(resp.content)

In [ ]:
spreadsheet = Spreadsheet("spreadsheet.xlsx")

In [ ]:
spreadsheet.del_worksheet('email-classification')
spreadsheet.del_worksheet('order-status')
spreadsheet.del_worksheet('order-response')
spreadsheet.del_worksheet('inquiry-response')
spreadsheet.add_worksheet('email-classification', ['Request ID', 'Email ID', 'Category'])
spreadsheet.add_worksheet('order-status', ['Request ID', 'Email ID', 'Product ID', 'Quantity', 'Status'])
spreadsheet.add_worksheet('order-response', ['Request ID', 'Email ID', 'Response'])
spreadsheet.add_worksheet('inquiry-response', ['Request ID', 'Email ID', 'Response'])

In [ ]:
spreadsheet.as_dataframe()['inquiry-response']

In [ ]:
for df in spreadsheet.as_dataframe().values():
    display(df.head(3))

In [ ]:
dfs = spreadsheet.as_dataframe()
products_df = dfs['products']
emails_df = dfs['emails']

# Display debugging
display(products_df)
display(emails_df.head(10))

In [ ]:
# Verify the product info
if len(products_df['product_id']) == len(products_df['product_id'].unique()):
    print(f"There are no duplicate products in inventory")
else:
    raise ValueError("duplicate products exist in inventory")

In [ ]:
class Email(object):
    def __init__(self, key: str, subject: str, message: str):
        self.id = key
        self.subject = subject if 'nan' not in str(subject) else "<NULL>"
        self.message = message

In [ ]:
# product class to hold product infomation
class Product(object):
    def __init__(self, key: str, name: str, category: str, info: str, stock: int, seasons: str, price: str):
        self.id = key
        self.name = name
        self.category = category
        self.description = info
        self.stock = int(stock)
        self.seasons: list[str] = [re.findall( r'\b\w+\b', season)[0] for season in seasons.split()]
        self.price = float(price)

    def create_order(self, amount: Union[int, None]):
        if amount is None or amount < -1 or amount == 0:
            raise ValueError(f"Invalid amount value '{amount}'")
        elif amount == -1:
            amount = self.stock
        rem = self.stock - amount
        if rem >= 0 and amount != 0:
            self.stock -= amount
            return amount
        elif self.stock == 0:
            return 0
        elif rem < 0:
            return -amount

    def get_info(self):
        return f"{self.name}{(self.category, self.description, self.seasons, self.price, self.stock)}".replace("'", "")

In [ ]:
class RequestType(Enum):
    Inquiry = 1
    Order = 2

# A request class to fulfill hold info requests
class Request(object):
    def __init__(self, key: str, product: Union[Product, None], email: Email, time: int, type: RequestType):
        self.id = key
        self.product = product
        self.email = email
        self.time = time
        self.type = type
        self.successful = False
        self.result: str = None
        self.responded = False

In [ ]:
# An inventory class to retrieve info on products and fulfill requests on orders and inquiries
class Inventory(object):
    request_id = -1
    def __init__(self, products: list[Product]):
        self.products: dict[str, Product] = {p.id: p for p in products}
        self.requests: dict[str, Request] = {}

    def _get_request_id(self):
        self.request_id += 1
        return self.request_id

    def _get_product(self, id: str):
        if id in self.products:
            return self.products[id]
        else:
            return None

    def _request(self, product_id: str, email: Email):
        product = self._get_product(product_id)
        request = Request(f"RQ{self._get_request_id()}", product, email, clock.time(), RequestType.Order)
        spreadsheet.add_record('email-classification', (request.id, email.id, request.type.name))
        return product, request

    def inquire(self, id: str, name: str, email: Email):
        product, request = self._request(id, email)
        if product:
            request.successful = True
            request.result = f"'{product.name}'(#{product.id}): {product.description}"
        else:
            request.result = f"The product '{name}' you are inquiring does not exist!"
        self.requests[request.id] = request
        spreadsheet.add_record('inquiry-response', (request.id, email.id, request.result))
        return request

    def order(self, id: str, name: str, amount: Union[int, None], email: Email):
        product, request = self._request(id, email)
        status = 'Out of Stock'
        if product:
            order_amount = product.create_order(amount)
            if order_amount > 0:
                request.successful = True
                bill = order_amount * product.price
                request.result = f"Successfully ordered {order_amount} of '{product.name}'(#{product.id}) for ${bill}"
                status = 'Completed'
            elif order_amount == 0:
                request.result = f"'{product.name}'(#{product.id}), is out of stock"
            else:
                request.result = f"Not enough stock to fill order of {product.name}'(#{product.id}); " \
                                 f"current stock={product.stock}, order amount={amount}"
        else:
            request.result = f"The product '{name}', you are ordering does not exist!"
        request.result = request.result.replace("'", "")
        self.requests[request.id] = request
        spreadsheet.add_record('order-status', (request.id, email.id, product.id if product else None, amount, status))
        spreadsheet.add_record('order-response', (request.id, email.id, request.result))
        return request

    def get_available_products(self):
        return [(key, p.name) for key, p in self.products.items()]

    def get_product_info(self):
        text = "With formatting ID ~ Name(Category, Description, Seasons, Price in $, Stock), product info:"
        for key, p in self.products.items():
            text += f"\n{key} ~ {p.get_info()}"
        return text

In [ ]:
inventory = Inventory([Product(*tuple(row)) for _, row in products_df.iterrows()])

for e, v in vars(random.choice(list(inventory.products.values()))).items():
    print(f"{e} -> {v} (type={type(v)})")

In [ ]:
print(inventory.get_product_info())

In [ ]:
MAX_TOKEN_NUM = int(4096 * 0.9)
MAX_TOKEN_NUM

# Task 1. Classify emails

In [ ]:
EMAILS = [Email(*tuple(row)) for _, row in emails_df.iterrows()]
EMAILS: dict[str, Email] = {email.id: email for email in EMAILS}
for e, v in vars(random.choice(list(EMAILS.values()))).items():
    print(f"{e} -> {v}")


def approximate_tokens(text: str):
    # Finding words
    words = len(text.split())
    # Finding symbols
    symbols = len(re.findall(r'[^\w\s]', text))
    return words + symbols
approximate_tokens("Hello there! How're you doing? My name is <FILL>")


# Assuming email has been processed into an instance on class Email
def process_email(email: Email):
    prompt = f"Email ID '{email.id}', ref: {email.subject}, msg: {email.message}"
    tokens = approximate_tokens(prompt)
    return prompt, tokens

# Batch process emails to fill as many tokens per prompt and improve on response speed
# This will save on the tokens used as you'd need to append a new template tokens for each email request if not using batches
def batch_process_emails(emails: list[Email], token_limit: int = None):
    if token_limit is None:
        token_limit = MAX_TOKEN_NUM
    batches = []
    batch = ""
    batch_idx = 0
    token_num = 0
    for i, email in enumerate(emails):
        prompt, tokens = process_email(email)
        if token_num + tokens + 3 > token_limit:
            if len(batch) > 0:
                batches.append(batch)
            print(batch)
            batch = ""
            batch_idx = 0
            token_num = 0
        batch += f"\n{batch_idx} => {prompt}"
        batch_idx += 1
        token_num += (tokens + 3)
        # print(batch)
        if len(batches) == 0 and i == len(emails)-1:
            batches.append(batch)
    return batches
display(f"-------------------")
print(batch_process_emails(list(EMAILS.values()))[0])
display(len(batch_process_emails(list(EMAILS.values()))))


def get_request_types():
    return [(k, v.value) for k, v in vars(RequestType).items() if "_" not in k]

display(f"-------------------")
print(get_request_types())


def classify_emails(emails: list[Email], max_attempts=10):
    # using RAG techniques
    products = str(inventory.get_available_products()).replace("'", "")
    request_types = str(get_request_types()).replace("'", "")
    template = f"Given the following list of available products and their product_id's, {products}." \
               f"and the enum values of the request types, {request_types}"  \
               f"\nONLY reply (in a string format) with a generated Python list of tuples, format" \
               f"(email_id, product_id, product_name, amount, request_type_value), classifying ALL the following email messages " \
               f"into request types 'Inquiries' and 'Orders',  based on the tone and content of the emails. " \
               f"\nNote that if product or product_id is not available the product_id set SHOULD be NULL only if the message reads like the email user was requesting something on the particular product." \
               f"\nNote that emails with multiple product requests MUST be split into multiples tuples for each requests." \
               f"\nNote that if the entire stock of an item is required, the amount value should be 'all' and 'NULL' when it is an  inquiry" \
               f"\nENSURE that no 'NULL' should be in amount value for orders" \
               f"\nDO NOT use an apostrophe to represent the strings" \
               f"\nNote that the tuples MUST be in new lines:"

    def filter_requests(response_):
        def split_words(line: str):
            split: list[str] = []
            for value in line.split():
                match = re.findall(r'\b\w+\b', str(value))
                if len(match) > 0:
                    split.append(match[0])
            return split

        def get_request_type(value: int):
            value = int(value)
            types = {v.value: v for k, v in vars(RequestType).items() if "_" not in k}
            if value in types:
                return types[value]
            else:
                raise ValueError(f"Unknown request type '{value}'")
            pass

        def str_combine(strings: list[str]):
            if len(strings) > 1:
                text = strings[0]
                for string in strings[1:]:
                    text += f" {string}"
            elif len(strings) == 1:
                text = strings[0]
            else:
                raise ValueError(f"Cannot combine empty list of strings")
            return text

        def get_amount(value: str):
            if 'all' in value.lower():
                return -1
            elif value.isnumeric():
                return int(value)
            elif value.lower() in ['none', 'null']:
                return None
            else:
                # raise ValueError(f"Unknown amount value {value}")
                return None

        # Filter the tuples from the string
        validation_pattern_1 = r'.*\(.*E.*\).*'
        validation_pattern_2 = r'.*\[.*E.*\].*'
        requests_: list[str] = [line for line in response_.content.replace(",", "\t").splitlines() if bool(re.match(validation_pattern_1, line)) or bool(re.match(validation_pattern_2, line))]
        requests_: list[str] = [line for line in requests_ if len(line) >= 4]
        # Filter the elements of the tuple
        requests_: list[list[str]] = [tuple(split_words(line)) for line in requests_]
        # Convert string tuple to an actual tuple
        requests_: list[tuple] = [(req[0], req[1], str_combine(req[2:len(req)-2]), get_amount(req[-2]), get_request_type(req[-1])) for req in requests_]
        # Filter the faulty tuples
        requests_ = [req for req in requests_ if not('null' in req[2].lower() or 'none' in req[2].lower())]
        return requests_

    responses, requests = [], []

    def get(text: str):
        prompt = template + text
        # print(prompt)

        client = OpenAI(
            base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
            api_key='a0BIj000001iX7PMAU'
        )

        completion = client.chat.completions.create(
          model="gpt-4o",
          messages=[
            {"role": "user", "content": prompt}
          ]
        )
        return completion.choices[0].message

    for batch in batch_process_emails(emails):
        done = False
        attempts_done = 0
        response, reqs = None, None
        while not (done or attempts_done >= max_attempts):
            response = get(batch)
            print(response.content)
            reqs = filter_requests(response)
            for req in reqs:
                if req[-1] == RequestType.Order and not isinstance(req[-2], int):
                    attempts_done += 1
                    continue
            done = True

        responses.append(response)
        requests += reqs

    # Filter request types
    inquiries = [req for req in requests if req[-1] == RequestType.Inquiry]
    orders = [req for req in requests if req[-1] == RequestType.Order]

    return responses, inquiries, orders


inventory.requests = {}
responses, INQUIRIES, ORDERS = classify_emails(list(EMAILS.values()))


for r_type in [INQUIRIES, ORDERS]:
    display(f"-------------------")
    for t in r_type:
        print(t)
    display(f"-------------------")

# Task 2. Process order requests

In [ ]:
# Execute order
# If you encounter 'None' error here, rerun Task 1 Cells
for order in ORDERS:
    email_id, product_id, product, amount, product_type = order
    inventory.order(product_id, product, amount, EMAILS[email_id])

# Task 3. Handle product inquiry

In [ ]:
# Execute inquiries
for inquiry in INQUIRIES:
    email_id, product_id, product, _, product_type = inquiry
    inventory.inquire(product_id, product, EMAILS[email_id])

# DEBUGGING

In [ ]:
def reply_email(request: Request):
    # Pseudo reply to email
    if not request.responded:
        print(f"For email {request.email.id} ~ #{request.id}: {request.result}")
        request.responded = True


def batch_reply_email(inventory: Inventory):
    for request in inventory.requests.values():
        reply_email(request)

In [ ]:
batch_reply_email(inventory)

# RESET responses for testing purposes
for request in inventory.requests.values():
    request.responded = False

In [250]:
for df_name, df in spreadsheet.as_dataframe().items():
    print(f"Sheet Name = {df_name}")
    display(df.head())

Sheet Name = products


,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1.0,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5.0,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6.0,"Spring, Fall",23.00
3,CSH1098,Cozy Shawl,Accessories,Wrap yourself in comfort with our cozy shawl. ...,3.0,"Fall, Winter",22.00
4,CHN0987,Chunky Knit Beanie,Accessories,Keep your head toasty with our chunky knit bea...,2.0,"Fall, Winter",22.00


Sheet Name = emails


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In..."
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc..."


Sheet Name = email-classification


...

,Request ID,Email ID,Product ID,Quantity,Status
0,RQ0,E001,LTH0976,-1,Completed
1,RQ1,E002,VBT2345,1,Completed
2,RQ2,E004,SFT1098,4,Completed
3,RQ3,E007,CLF2109,5,Out of Stock
4,RQ4,E007,FZZ1098,2,Completed


Sheet Name = order-response


,Request ID,Email ID,Response
0,RQ0,E001,Successfully ordered 4 of Leather Bifold Walle...
1,RQ1,E002,Successfully ordered 1 of Vibrant Tote(#VBT234...
2,RQ2,E004,Successfully ordered 4 of Infinity Scarf(#SFT1...
3,RQ3,E007,Not enough stock to fill order of Cable Knit B...
4,RQ4,E007,Successfully ordered 2 of Fuzzy Slippers(#FZZ1...


Sheet Name = inquiry-response


,Request ID,Email ID,Response
0,RQ11,E003,'Leather Backpack'(#LTH1098): Upgrade your dai...
1,RQ12,E003,'Leather Tote'(#LTH5432): Elevate your everyda...
2,RQ13,E005,'Cozy Shawl'(#CSH1098): Wrap yourself in comfo...
3,RQ14,E006,'Chelsea Boots'(#CBT8901): Elevate your casual...
4,RQ15,E009,'Chunky Knit Beanie'(#CHN0987): Keep your head...
